# ANEETA Evaluation: Local RAG and Model Comparisons
This notebook provides a reproducible baseline evaluation of the ANEETA Doubt Solver using local models via Ollama and the existing ANEETA vector stores. It also verifies repository setup and supports simple model comparisons for group reporting.

In [1]:
# 1) Verify Local Repos and Paths
import os, sys, subprocess, shutil
from pathlib import Path
ROOT = Path.cwd().resolve()
# This notebook lives INSIDE the ANEETAA repo
aneeta_dir = ROOT  # current directory IS the ANEETAA repo
print("Current working dir:", ROOT)
print("ANEETAA repo at:", aneeta_dir)

# Verify we have the key ANEETAA directories
required_dirs = ["src", "Processed Data", "Raw Data"]
for d in required_dirs:
    if (aneeta_dir / d).exists():
        print(f"✓ Found: {d}")
    else:
        print(f"✗ Missing: {d}")

print("Repo verified.")

Current working dir: D:\Git Projects\SMU\ANEETAA
ANEETAA repo at: D:\Git Projects\SMU\ANEETAA
✓ Found: src
✓ Found: Processed Data
✓ Found: Raw Data
Repo verified.


In [2]:
# 2) Create and Validate Python Environment
import platform, sys, subprocess, os
from pathlib import Path
print("Python:", sys.version)
assert sys.version_info >= (3,10), "Python >= 3.10 required"
print("Executable:", sys.executable)
print("Platform:", platform.platform())

# Optionally create venv instructions (not auto-creating to avoid permission issues)
venv_dir = ROOT / ".venv"
print("Suggested venv path:", venv_dir)
print("If you need one, run in PowerShell:")
print("python -m venv .venv; .\\.venv\\Scripts\\Activate.ps1; python -m pip install -U pip")

# CUDA check (optional)
try:
    import torch
    print("Torch version:", torch.__version__)
    print("CUDA available:", torch.cuda.is_available())
except Exception as e:
    print("Torch not installed or CUDA not available:", e)

Python: 3.11.8 (tags/v3.11.8:db85d51, Feb  6 2024, 22:03:32) [MSC v.1937 64 bit (AMD64)]
Executable: c:\Users\benja\AppData\Local\Programs\Python\Python311\python.exe
Platform: Windows-10-10.0.26200-SP0
Suggested venv path: D:\Git Projects\SMU\ANEETAA\.venv
If you need one, run in PowerShell:
python -m venv .venv; .\.venv\Scripts\Activate.ps1; python -m pip install -U pip
Torch not installed or CUDA not available: No module named 'torch'


In [3]:
# 3) Install and Import Dependencies (lean and fast on Windows)
import sys, subprocess, os

# Speed: upgrade pip tooling first
subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "pip", "setuptools", "wheel"], check=True)

# Minimal core deps; avoid heavy extras here
# Use binary wheels where possible to speed up installs
packages = [
    "typing_extensions>=4.12.2",
    "pydantic<2.0.0",        # compatible with chromadb 0.4.x
    "numpy==1.26.4",         # stable wheels
    "pandas==2.2.2",         # pin to a wheel-friendly version
    "requests",
    "tqdm>=4.65.0",
    "jsonlines",
    "chromadb==0.4.24",      # no ONNX default embedder
    # Minimal ANEETA runtime deps (pure-Python)
    "langchain", "langchain-community", "langchain-ollama", "langgraph"
]

print("Installing core packages (binary wheels preferred)...")
subprocess.run([sys.executable, "-m", "pip", "install", "--only-binary", ":all:", *packages], check=False)

# Imports (defer MLflow to services cell and make it optional)
import chromadb, pandas as pd, numpy as np, requests  # noqa: F401
print("Core imports OK (lean setup)")

Installing core packages (binary wheels preferred)...
Core imports OK (lean setup)
Core imports OK (lean setup)


In [4]:
# 4) Configure Providers (Ollama first; optional OpenAI)
import os, json, requests
from dataclasses import dataclass
OLLAMA_URL = os.getenv("OLLAMA_URL", "http://localhost:11434")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

@dataclass
class Provider:
    kind: str  # 'ollama' or 'openai'
    model: str

def get_available_models():
    try:
        r = requests.get(f"{OLLAMA_URL}/api/tags", timeout=5)
        if r.ok:
            return {m.get('name') for m in r.json().get('models', []) if 'name' in m}
    except Exception:
        pass
    return set()

def pick_provider(prefer_ollama: bool=True) -> Provider:
    if prefer_ollama:
        installed = get_available_models()
        # Default order for NEET evaluation: llama3.1:8b → gemma2:9b → mistral-nemo:12b
        order = ['llama3.1:8b', 'gemma2:9b', 'mistral-nemo:12b']
        for m in order:
            if m and ((not installed) or (m in installed)):
                return Provider('ollama', m)
    if OPENAI_API_KEY:
        return Provider('openai', 'gpt-4o-mini')
    # Fallback
    return Provider('ollama', 'llama3.1:8b')

provider = pick_provider(True)
print("Using provider:", provider)

Using provider: Provider(kind='ollama', model='llama3.1:8b')


In [5]:
# 5) Start Services: Ollama, optional MLflow, ChromaDB
import time, requests, os, subprocess, sys, pathlib
from pathlib import Path

# Silence Chroma telemetry noise (set before importing chromadb)
os.environ.setdefault("ANONYMIZED_TELEMETRY", "false")
os.environ.setdefault("CHROMA_TELEMETRY_ENABLED", "false")


def check_ollama(url=OLLAMA_URL):
    try:
        r = requests.get(url)
        return r.status_code in (200, 404)  # 404 is OK for root
    except Exception:
        return False


if not check_ollama():
    print("Ollama not reachable at", OLLAMA_URL)
    print("- Install/start Ollama, then run: ollama serve")
else:
    print("Ollama reachable at", OLLAMA_URL)
    print("Pull these tags for the evaluations (pick any/all):")
    print(" - ollama pull llama3.1:8b   # tools-capable local chat model (baseline)")
    print(" - ollama pull gemma2:9b     # higher quality candidate (more RAM)")
    print(" - ollama pull mistral-nemo:12b  # higher quality candidate (heavier)")
    print(" - ollama pull nomic-embed-text  # embeddings needed for vector DB")

# Optional MLflow setup (skip if not installed)
try:
    import mlflow  # noqa: F401
    mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI", "http://127.0.0.1:5000"))
    print("MLflow tracking URI:", mlflow.get_tracking_uri())
    MLFLOW_OK = True
except Exception as e:
    print("MLflow not installed/available (optional):", e)
    MLFLOW_OK = False

# Initialize a persistent Chroma client, reusing any existing kernel client to avoid settings conflicts
import chromadb
persist_dir = str((ROOT / ".chroma").resolve())

reuse = False
if "client" in globals() and getattr(globals().get("client"), "get_or_create_collection", None):
    # Reuse existing client to avoid "already exists with different settings" errors
    print("Reusing existing ChromaDB client. Persisting at:", persist_dir)
    reuse = True

if not reuse:
    try:
        from chromadb.config import Settings
        settings = Settings(persist_directory=persist_dir, anonymized_telemetry=False)
        client = chromadb.Client(settings)
        print("ChromaDB client created via Settings. Persisting at:", persist_dir)
    except Exception as e:
        print("Settings-based client failed, falling back to PersistentClient:", e)
        client = chromadb.PersistentClient(path=persist_dir)
        print("ChromaDB PersistentClient created. Persisting at:", persist_dir)

try:
    _ = client.get_or_create_collection("smoke_test")
    print("ChromaDB OK.")
except ValueError as e:
    if "already exists" in str(e):
        print("Chroma singleton already initialized with different settings; reusing existing client.")
    else:
        print("ChromaDB init failed:", e)
        raise

Ollama reachable at http://localhost:11434
Pull these tags for the evaluations (pick any/all):
 - ollama pull llama3.1:8b   # tools-capable local chat model (baseline)
 - ollama pull gemma2:9b     # higher quality candidate (more RAM)
 - ollama pull mistral-nemo:12b  # higher quality candidate (heavier)
 - ollama pull nomic-embed-text  # embeddings needed for vector DB
MLflow not installed/available (optional): No module named 'mlflow'


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


ChromaDB client created via Settings. Persisting at: D:\Git Projects\SMU\ANEETAA\.chroma
ChromaDB OK.


In [6]:
# HOTFIX: ensure jsonlines is available
import sys, subprocess
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "jsonlines"], check=True)
import jsonlines; print("jsonlines installed and imported")

jsonlines installed and imported


### Troubleshooting Ollama model pulls (Windows)
If `ollama pull` fails with `pull model manifest: file does not exist`:

1) Confirm the service is running
- In a new PowerShell: `ollama serve` (if you see `bind: Only one usage...`, it means Ollama is already running; skip this.)
- Check: `Invoke-WebRequest http://localhost:11434/ -UseBasicParsing`

2) Use valid tags for your version
- Try: `ollama pull llama3.1:8b`  
- Or: `ollama pull gemma2:9b`
- Or: `ollama pull mistral-nemo:12b`

3) Update Ollama
- Download the latest from https://ollama.com/download and reinstall.

4) Clear local cache (optional)
- Stop service; delete `%LOCALAPPDATA%\Ollama\models` cautiously; then `ollama serve` and pull again.

Note: For ANEETA Doubt Solver, you DO need the embedding model `nomic-embed-text` to open the existing Chroma vector stores. Pull it with: `ollama pull nomic-embed-text`. Also prefer a tools-capable chat model like `llama3.1:8b`.

In [7]:
# 6) Load ANEETA Dataset → normalize to DataFrame (repo data only)
import pandas as pd, json, os, re
from pathlib import Path

# Local convenience path (optional)
local_processed = (ROOT / "datasets" / "aneeta" / "processed").resolve()
# Authoritative repo path
repo_processed = (aneeta_dir / "Processed Data").resolve()

rows = []
source_used = None

# 1) Prefer local test.jsonl if present (explicit test set)
test_path = local_processed / "test.jsonl"
if test_path.exists():
    import jsonlines
    with jsonlines.open(test_path, 'r') as reader:
        rows = list(reader)
    source_used = str(test_path)

# Helper: specialized parser for NEET solved papers (extract Q and numeric answer)
def parse_solved_mcqs(json_path: Path):
    try:
        data = json.loads(json_path.read_text(encoding='utf-8'))
    except Exception:
        return []
    # Concatenate pages and search for patterns like: "1. <question/body> ... Answer (3)"
    text = "\n\n".join([str(obj.get("page_content", "")) for obj in data])
    text = text.replace("\r", "")
    # Regex: capture question number, question block (non-greedy), and the numeric answer in parentheses
    pat = re.compile(r"(?:^|\n)\s*(\d{1,3})\.\s*(.*?)(?:\n\s*Answer\s*\((\d+)\))", re.S)
    out = []
    used_ids = set()
    for i, m in enumerate(pat.finditer(text)):
        qn = m.group(1) or str(i)
        body = (m.group(2) or "").strip()
        # Truncate at a solution marker if present to keep the stem/options only
        body = body.split("\nSol.")[0].strip()
        # Normalize whitespace
        body = re.sub(r"[ \t]+", " ", body)
        ans_num = m.group(3)
        # Build a stable id even if question numbers repeat across sections
        uid = f"neet:{qn}:{i}"
        if uid in used_ids:
            continue
        used_ids.add(uid)
        # Keep the whole body as the question (includes options like (1)...(4) when present)
        out.append({
            "id": uid,
            "question": body,
            "context": "",
            "answer": f"({ans_num})"
        })
    return out

# 2) Otherwise, strictly load from ANEETA repo's Processed Data
if not rows:
    if not repo_processed.exists():
        raise SystemExit(f"ANEETA processed data folder not found at: {repo_processed}")
    print("Using ANEETA repo processed data at:", repo_processed)

    # First, try specialized parse from solved_question_papers.json for MCQ Q/A pairs
    solved_path = repo_processed / "solved_question_papers.json"
    if solved_path.exists():
        parsed = parse_solved_mcqs(solved_path)
        if parsed:
            rows = parsed
            source_used = str(solved_path)

    # If still empty, fall back to generic extraction from other processed files (context only)
    if not rows:
        candidates = [
            "mentor_data.json",
            "processed_biology_chunks.json",
            "processed_chemistry_chunks.json",
            "processed_physics_chunks.json",
        ]
        def load_json(path: Path):
            try:
                data = json.loads(path.read_text(encoding='utf-8'))
                if isinstance(data, dict):
                    # flatten dict-of-lists if needed
                    flat = []
                    for v in data.values():
                        if isinstance(v, list):
                            flat.extend(v)
                    return flat or [data]
                return data if isinstance(data, list) else [data]
            except Exception:
                return []
        def get_ci(d: dict, keys: list[str]):
            # case-insensitive get
            lk = {k.lower(): k for k in d.keys()}
            for k in keys:
                if k.lower() in lk:
                    return d.get(lk[k.lower()])
            return None
        extracted = []
        used_file = None
        for name in candidates:
            f = repo_processed / name
            if not f.exists():
                continue
            data = load_json(f)
            for i, rec in enumerate(data):
                if not isinstance(rec, dict):
                    continue
                # These processed files generally have 'page_content' text chunks
                ctx = get_ci(rec, ["page_content","content","text","chunk"]) or ""
                if isinstance(ctx, (dict, list)):
                    ctx = json.dumps(ctx, ensure_ascii=False)
                if isinstance(ctx, str) and len(ctx.strip()) >= 20:
                    # Use the chunk as context and prompt the agent with a generic instruction
                    q_text = ctx.strip().split("\n")[0]
                    extracted.append({
                        "id": f"{name}:{i}",
                        "question": q_text[:500],
                        "context": ctx[:2000],
                        "answer": ""
                    })
            if extracted:
                rows = extracted
                used_file = f
                break
        source_used = str(used_file) if used_file else source_used

# Build DataFrame
import pandas as pd

df = pd.DataFrame(rows)
if df.empty:
    raise SystemExit("Empty dataset after parsing ANEETA data.")
print("Loaded rows:", len(df), "from:", source_used)

# Normalize columns and order
df = df.rename(columns={"sample_id":"id"})
for col in ["id","question","context","answer"]:
    if col not in df.columns:
        df[col] = None
# Cap dataset size for quick iteration
max_n = int(os.getenv("ANEETA_MAX_RECORDS", "250"))
df = df[["id","question","context","answer"]].head(max_n)
df.head(3)

Using ANEETA repo processed data at: D:\Git Projects\SMU\ANEETAA\Processed Data
Loaded rows: 8890 from: D:\Git Projects\SMU\ANEETAA\Processed Data\solved_question_papers.json
Loaded rows: 8890 from: D:\Git Projects\SMU\ANEETAA\Processed Data\solved_question_papers.json


,id,question,context,answer
0,neet:1:0,Two objects of mass 10 kg and 20 kg respective...,,(3)
1,neet:2:1,Match List-I with List-II \n List-I \n(Electro...,,(1)
2,neet:3:2,The energy that will be ideally radiated by a ...,,(2)


## Baseline ANEETA Doubt Solver (non‑DSPy)
The following cells call ANEETA's mcq_question_solver_agent through the LangGraph workflow to provide a baseline Doubt Solver run without DSPy.

In [8]:
# Configure ANEETA environment (local-only)
import os, sys, re

# Force baseline model for NEET evaluation (override any environment defaults)
os.environ["LLM_MODEL"] = "llama3.1:8b"
os.environ["CREATIVE_LLM_MODEL"] = "llama3.1:8b"
# Embeddings are required to open the persisted Chroma vector stores; use the model that matches the DBs
os.environ.setdefault("EMBEDDING_MODEL", "nomic-embed-text")
# Use the ANEETA repo's persisted Chroma DBs
os.environ.setdefault("VECTORDB_BASE_PATH", str((aneeta_dir / "src" / "aneeta" / "vectordb").resolve()))

print("LLM_MODEL:", os.environ["LLM_MODEL"]) 
print("CREATIVE_LLM_MODEL:", os.environ["CREATIVE_LLM_MODEL"]) 
print("EMBEDDING_MODEL:", os.environ["EMBEDDING_MODEL"])  # tip: if missing locally, run: ollama pull nomic-embed-text
print("ANEETA VECTORDB_BASE_PATH:", os.environ["VECTORDB_BASE_PATH"])

# Add ANEETA src to path for imports
sys.path.insert(0, str((aneeta_dir / "src").resolve()))
print("PYTHONPATH +:", (aneeta_dir/"src").resolve())

# Hard reset: clear Streamlit cache and fully purge ANEETA modules so model changes take effect
import importlib
try:
    import streamlit as st
    try:
        st.cache_resource.clear()
        print("Cleared streamlit cache_resource")
    except Exception:
        pass
except Exception:
    st = None  # type: ignore

# Purge ALL aneeta modules to avoid stale singletons (llm, resources, workflow, etc.)
for mod in list(sys.modules.keys()):
    if mod == "aneeta" or mod.startswith("aneeta."):
        sys.modules.pop(mod, None)

# Import in a clean order: LLM -> resources -> agents/router -> workflow
import aneeta.llm.llm as llm_mod
importlib.reload(llm_mod)

import aneeta.core.resources as resources_mod
importlib.reload(resources_mod)

import aneeta.nodes.agents as agents_mod
import aneeta.nodes.router as router_mod
importlib.reload(agents_mod)
importlib.reload(router_mod)

import aneeta.graph.workflow as workflow_mod
importlib.reload(workflow_mod)

# Print resolved model from ANEETA resources for verification
try:
    from aneeta.core.resources import llm as current_llm
    resolved_model = getattr(current_llm, "model", None) or getattr(current_llm, "model_name", None) or getattr(current_llm, "model_id", None)
    print("ANEETA resources -> llm.model:", resolved_model)
except Exception as e:
    print("Could not read ANEETA resources llm model:", e)

# Define a simple whitespace normalizer to clean streamed outputs
def normalize_whitespace(text: str) -> str:
    t = text.replace("\r", "")
    # Collapse 3+ newlines to 2
    t = re.sub(r"\n{3,}", "\n\n", t)
    # Collapse runs of spaces
    t = re.sub(r" {2,}", " ", t)
    return t.strip()

# Define baseline solve_mcq using ANEETA LangGraph
from langchain_core.messages import HumanMessage

def solve_mcq(question: str, timeout: int = 120, language: str | None = None):
    try:
        graph = workflow_mod.get_graph()
        init_state = {
            "messages": [HumanMessage(content=question)],
            "user_explanation_language": language or os.getenv("ANEETA_EXPLANATION_LANG", "English"),
        }
        # Stream and collect final text (supports either messages or response_stream)
        final_text = []
        for update in graph.stream(init_state):
            for _, v in update.items():
                if isinstance(v, dict):
                    # If agent provided a streaming generator, consume it
                    rs = v.get("response_stream")
                    if rs is not None:
                        try:
                            for chunk in rs:
                                # Don't strip() here to preserve word boundaries
                                t = str(getattr(chunk, "content", chunk))
                                if t:
                                    final_text.append(t)
                        except Exception:
                            pass
                    # Also collect from messages if present
                    if "messages" in v:
                        msgs = v.get("messages") or []
                        if msgs and hasattr(msgs[-1], "content"):
                            final_text.append(str(msgs[-1].content))
        # Join with empty string to avoid breaking words
        raw_text = "".join(final_text).strip()
        return {"text": normalize_whitespace(raw_text)}
    except Exception as e:
        return {"text": f"[baseline error: {e}]"}

# Smoke test (removed [:300] truncation to show full output)
test_q = "Which organelle is the powerhouse of the cell? Options: (A) Ribosome (B) Mitochondria (C) Golgi (D) Lysosome"
print("Smoke test:")
print(solve_mcq(test_q).get("text", ""))

LLM_MODEL: llama3.1:8b
CREATIVE_LLM_MODEL: llama3.1:8b
EMBEDDING_MODEL: nomic-embed-text
ANEETA VECTORDB_BASE_PATH: src/aneeta/vectordb
PYTHONPATH +: D:\Git Projects\SMU\ANEETAA\src
Cleared streamlit cache_resource
Cleared streamlit cache_resource


2025-10-31 00:25:21.785 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 00:25:22.117 
  command:

    streamlit run C:\Users\benja\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-10-31 00:25:22.117 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 00:25:22.118 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 00:25:22.118 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 00:25:22.117 
  command:

    streamlit run C:\Users\benja\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-10-31 00:25:22.117 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
202

ANEETA resources -> llm.model: llama3.1:8b
Smoke test:
Answer: (B) Mitochondria

Explanation: The mitochondria are known as the "powerhouse" of the cell because they generate most of the cell's supply of adenosine triphosphate (ATP), which is used as a source of chemical energy. In addition to generating ATP, mitochondria are involved in a range of other processes, including signaling, cellular differentiation, cell death, as well as the control of the cell cycle and cell growth and division.

Translation: माइटोकॉन्ड्रिया कोशिका का शक्ति केंद्र है।
Answer: (B) Mitochondria

Explanation: The mitochondria are known as the "powerhouse" of the cell because they generate most of the cell's supply of adenosine triphosphate (ATP), which is used as a source of chemical energy. In addition to generating ATP, mitochondria are involved in a range of other processes, including signaling, cellular differentiation, cell death, as well as the control of the cell cycle and cell growth and division.

T

In [9]:
# Quick model switch micro-benchmark (local)
# This is optional; uses installed models only
try:
    r = requests.get(f"{OLLAMA_URL}/api/tags", timeout=10)
    names = [m.get('name') for m in r.json().get('models', [])]
except Exception:
    names = []

# Shortlist: baseline + two higher-quality candidates
model_shortlist = ["llama3.1:8b", "gemma2:9b", "mistral-nemo:12b"]
preferred = [m for m in model_shortlist if (not names) or (m in names)]
if not preferred:
    preferred = ["llama3.1:8b"]  # safe default

benchmarks = []
for name in preferred:
    os.environ["LLM_MODEL"] = name
    s = time.time(); _ = solve_mcq("During DNA replication, the enzyme responsible for joining Okazaki fragments is: Options: (A) DNA polymerase (B) DNA ligase (C) Helicase (D) Primase"); dt = (time.time()-s)*1000
    benchmarks.append({"model": name, "latency_ms": dt})

pd.DataFrame(benchmarks).sort_values("latency_ms").reset_index(drop=True)

,model,latency_ms
0,llama3.1:8b,2855.539799
1,mistral-nemo:12b,3219.932795
2,gemma2:9b,3339.609146


In [10]:
# Evaluate 3 MCQs across different local models with comprehensive metrics
import os, numpy as np, pandas as pd, time, requests, json, re
from pathlib import Path
try:
    from IPython.display import display
except Exception:
    display = lambda x: print(x)  # fallback

# Quick reachability check to avoid confusing errors
def _ollama_ok(url: str) -> bool:
    try:
        r = requests.get(url, timeout=5)
        return r.status_code in (200, 404)
    except Exception:
        return False

if not _ollama_ok(OLLAMA_URL):
    print("Ollama not reachable at", OLLAMA_URL)
    print("Start the service in a PowerShell:")
    print("ollama serve    # then rerun this cell")
else:
    # Helper: get available Ollama tags to filter the test set
    available = set()
    try:
        r = requests.get(f"{OLLAMA_URL}/api/tags", timeout=10)
        if r.ok:
            for m in r.json().get('models', []):
                if 'name' in m:
                    available.add(m['name'])
    except Exception:
        pass

    # Shortlist for one-click comparisons
    model_shortlist = ["llama3.1:8b", "gemma2:9b", "mistral-nemo:12b"]
    models_to_test = [m for m in model_shortlist if (not available) or (m in available)]
    missing = [m for m in model_shortlist if m not in models_to_test]
    if missing:
        print("Missing models not found in Ollama; pull as needed:")
        for m in missing:
            print(f" - ollama pull {m}")
    if not models_to_test:
        models_to_test = ["llama3.1:8b"]  # safe default

    # Load ground truth from ANEETA processed data
    print("Loading ground truth answers from ANEETA repository...")
    solved_papers_path = aneeta_dir / "Processed Data" / "solved_question_papers.json"
    ground_truth_map = {}
    
    if solved_papers_path.exists():
        try:
            data = json.loads(solved_papers_path.read_text(encoding='utf-8'))
            # Build a simple question -> answer mapping
            text = "\n\n".join([str(obj.get("page_content", "")) for obj in data])
            # Extract questions and their answers
            pattern = re.compile(r"(\d{1,3})\.\s*(.*?)(?:\n\s*Answer\s*\((\d+)\))", re.S)
            for match in pattern.finditer(text):
                q_num = match.group(1)
                q_body = match.group(2).strip().split("\nSol.")[0].strip()
                answer_num = match.group(3)
                # Store first 100 chars of question as key
                key = q_body[:100].lower()
                ground_truth_map[key] = f"({answer_num})"
            print(f"✓ Loaded {len(ground_truth_map)} ground truth Q&A pairs")
        except Exception as e:
            print(f"⚠ Could not load ground truth: {e}")
    else:
        print(f"⚠ Ground truth file not found at {solved_papers_path}")

    # NEET-aligned MCQ examples with ground truth answers
    mcq_examples = [
        {
            "subject": "Physics",
            "question": "A body moving in a circle of radius r with speed v has centripetal acceleration? Options: (A) v^2/r (B) r/v^2 (C) v/r^2 (D) r^2/v",
            "correct_answer": "(A)"  # v^2/r is correct
        },
        {
            "subject": "Chemistry", 
            "question": "If 2 mol of an ideal gas at constant temperature compress to half the volume, what happens to pressure? Options: (A) doubles (B) halves (C) same (D) zero",
            "correct_answer": "(A)"  # Boyle's law: P1V1 = P2V2, so P doubles
        },
        {
            "subject": "Biology",
            "question": "During DNA replication, the enzyme responsible for joining Okazaki fragments is: Options: (A) DNA polymerase (B) DNA ligase (C) Helicase (D) Primase",
            "correct_answer": "(B)"  # DNA ligase joins Okazaki fragments
        },
    ]

    # LLM Judge function (uses llama3.1:8b as judge)
    def judge_answer_quality(question: str, answer: str, judge_model: str = "llama3.1:8b") -> dict:
        """
        Uses an LLM to judge answer quality on clarity, reasoning steps, and explanation quality.
        Returns score 1-10 and reasoning.
        """
        try:
            from langchain_ollama import ChatOllama
            judge_llm = ChatOllama(model=judge_model, base_url=OLLAMA_URL)
            
            judge_prompt = f"""You are evaluating a student's answer to a NEET exam question. Rate the answer quality on a scale of 1-10 based on:
- Clarity of explanation (is it easy to understand?)
- Logical reasoning steps (does it show clear step-by-step thinking?)
- Correctness of approach (even if final answer is wrong, is the reasoning sound?)

Question: {question}

Student's Answer:
{answer}

Provide your evaluation in this exact format:
Score: [number 1-10]
Reasoning: [brief explanation of your score]

Your evaluation:"""
            
            response = judge_llm.invoke(judge_prompt)
            response_text = response.content if hasattr(response, 'content') else str(response)
            
            # Extract score
            score_match = re.search(r"Score:\s*(\d+)", response_text)
            score = int(score_match.group(1)) if score_match else 5
            score = max(1, min(10, score))  # Clamp to 1-10
            
            # Extract reasoning
            reasoning_match = re.search(r"Reasoning:\s*(.+)", response_text, re.S)
            reasoning = reasoning_match.group(1).strip()[:200] if reasoning_match else "No reasoning provided"
            
            return {"score": score, "reasoning": reasoning}
        except Exception as e:
            return {"score": 5, "reasoning": f"Judge error: {str(e)[:100]}"}

    # Fact-check function
    def fact_check_answer(model_answer: str, correct_answer: str) -> int:
        """
        Compares model answer against ground truth.
        Returns score 0-10 based on answer correctness.
        """
        # Extract the option letter from model answer (look for (A), (B), (C), (D), etc.)
        model_option = re.search(r"\(([A-D1-4])\)", model_answer)
        model_option = model_option.group(1) if model_option else None
        
        correct_option = re.search(r"\(([A-D1-4])\)", correct_answer)
        correct_option = correct_option.group(1) if correct_option else None
        
        if model_option and correct_option:
            return 10 if model_option == correct_option else 0
        else:
            # Fuzzy match: check if correct answer appears in response
            return 7 if correct_answer.lower() in model_answer.lower() else 2

    def eval_model_comprehensive(model_name: str):
        os.environ["LLM_MODEL"] = model_name
        results = []
        latencies = []
        
        for mcq in mcq_examples:
            print(f"  - {mcq['subject']}...", end=" ", flush=True)
            t0 = time.time()
            out = solve_mcq(mcq["question"])
            dt = (time.time()-t0)*1000
            latencies.append(dt)
            
            answer_text = out.get("text","")
            answer_summary = answer_text.split("\n")[0][:200] if answer_text else "[no response]"
            
            # Fact-check score
            fact_score = fact_check_answer(answer_text, mcq["correct_answer"])
            
            # Quality score (only judge if model produced an answer)
            if answer_text and len(answer_text) > 10:
                quality_result = judge_answer_quality(mcq["question"], answer_text)
                quality_score = quality_result["score"]
                quality_reasoning = quality_result["reasoning"]
            else:
                quality_score = 1
                quality_reasoning = "No meaningful answer provided"
            
            print(f"✓ (fact:{fact_score}/10, quality:{quality_score}/10, latency:{dt:.0f}ms)")
            
            results.append({
                "subject": mcq["subject"],
                "answer": answer_summary,
                "latency_ms": round(dt, 1),
                "fact_check_score": fact_score,
                "quality_score": quality_score,
                "quality_reasoning": quality_reasoning
            })
        
        return {
            "model": model_name,
            "avg_latency_ms": round(np.mean(latencies), 1),
            "avg_fact_score": round(np.mean([r["fact_check_score"] for r in results]), 1),
            "avg_quality_score": round(np.mean([r["quality_score"] for r in results]), 1),
            "results": results
        }

    # Evaluate all models
    all_results = []
    print("\n" + "="*60)
    print("Evaluating models on 3 NEET MCQs with comprehensive metrics")
    print("="*60 + "\n")
    
    for m in models_to_test:
        print(f"📊 Testing {m}:")
        eval_result = eval_model_comprehensive(m)
        all_results.append(eval_result)
        print()
    
    # Build detailed DataFrame
    rows = []
    for result in all_results:
        for i, mcq_result in enumerate(result["results"], 1):
            rows.append({
                "Model": result["model"],
                "Question": f"Q{i} ({mcq_result['subject']})",
                "Answer": mcq_result["answer"],
                "Latency_ms": mcq_result["latency_ms"],
                "Fact_Score": mcq_result["fact_check_score"],
                "Quality_Score": mcq_result["quality_score"],
                "Quality_Reasoning": mcq_result["quality_reasoning"]
            })
    
    df_detailed = pd.DataFrame(rows)
    
    # Summary DataFrame
    summary_rows = []
    for result in all_results:
        summary_rows.append({
            "Model": result["model"],
            "Avg_Latency_ms": result["avg_latency_ms"],
            "Avg_Fact_Score": result["avg_fact_score"],
            "Avg_Quality_Score": result["avg_quality_score"],
            "Overall_Score": round((result["avg_fact_score"] + result["avg_quality_score"]) / 2, 1)
        })
    
    df_summary = pd.DataFrame(summary_rows).sort_values("Overall_Score", ascending=False)
    
    # Optional MLflow logging
    try:
        import mlflow
        mlflow.set_experiment("aneeta-baseline-doubtsolver-comprehensive")
        for result in all_results:
            with mlflow.start_run(run_name=f"baseline_{result['model']}"):
                mlflow.log_params({"model": result["model"], "agent": "mcq_question_solver"})
                mlflow.log_metrics({
                    "avg_latency_ms": result["avg_latency_ms"],
                    "avg_fact_score": result["avg_fact_score"],
                    "avg_quality_score": result["avg_quality_score"]
                })
        print("✓ Logged to MLflow\n")
    except Exception:
        pass
    
    print("\n" + "="*60)
    print("SUMMARY: Model Performance Comparison")
    print("="*60)
    display(df_summary)
    
    print("\n" + "="*60)
    print("DETAILED: Per-Question Results")
    print("="*60)
    display(df_detailed[["Model", "Question", "Latency_ms", "Fact_Score", "Quality_Score"]])

Loading ground truth answers from ANEETA repository...
✓ Loaded 1938 ground truth Q&A pairs

Evaluating models on 3 NEET MCQs with comprehensive metrics

📊 Testing llama3.1:8b:
  - Physics... ✓ (fact:10/10, quality:9/10, latency:4849ms)
  - Chemistry... ✓ (fact:10/10, quality:9/10, latency:4849ms)
  - Chemistry... ✓ (fact:2/10, quality:9/10, latency:4987ms)
  - Biology... ✓ (fact:2/10, quality:9/10, latency:4987ms)
  - Biology... ✓ (fact:10/10, quality:9/10, latency:3133ms)

📊 Testing gemma2:9b:
  - Physics... ✓ (fact:10/10, quality:9/10, latency:3133ms)

📊 Testing gemma2:9b:
  - Physics... ✓ (fact:10/10, quality:9/10, latency:4262ms)
  - Chemistry... ✓ (fact:10/10, quality:9/10, latency:4262ms)
  - Chemistry... ✓ (fact:2/10, quality:9/10, latency:4587ms)
  - Biology... ✓ (fact:2/10, quality:9/10, latency:4587ms)
  - Biology... ✓ (fact:10/10, quality:9/10, latency:2837ms)

📊 Testing mistral-nemo:12b:
  - Physics... ✓ (fact:10/10, quality:9/10, latency:2837ms)

📊 Testing mistral-nemo:12

,Model,Avg_Latency_ms,Avg_Fact_Score,Avg_Quality_Score,Overall_Score
0,llama3.1:8b,4323.0,7.3,9.0,8.2
1,gemma2:9b,3895.3,7.3,9.0,8.2
2,mistral-nemo:12b,4409.8,7.3,8.7,8.0



DETAILED: Per-Question Results


,Model,Question,Latency_ms,Fact_Score,Quality_Score
0,llama3.1:8b,Q1 (Physics),4849.1,10,9
1,llama3.1:8b,Q2 (Chemistry),4986.8,2,9
2,llama3.1:8b,Q3 (Biology),3133.2,10,9
3,gemma2:9b,Q1 (Physics),4261.9,10,9
4,gemma2:9b,Q2 (Chemistry),4587.3,2,9
5,gemma2:9b,Q3 (Biology),2836.6,10,9
6,mistral-nemo:12b,Q1 (Physics),4698.2,10,8
7,mistral-nemo:12b,Q2 (Chemistry),5284.7,2,9
8,mistral-nemo:12b,Q3 (Biology),3246.6,10,9
